# A Demo For Building Attribute Hierarchy Based on [Animal with Attributes 2 Dataset](https://cvml.ist.ac.at/AwA2/) 😀



This is a demo of hierarchy building pipline metioned in paper "Hierarchical Visual Attribute Learning in the Wild" based on AWA2 dataset. If you want to execute this demo, please prepare a `OPENAI_API_KEY` for using ChatGPT.

## Install and import package

In [1]:
!pip install langchain openai

Store the OPENAI_API_KEY into your environment

In [2]:
import os
import openai
import json
from pprint import pprint

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

os.environ["OPENAI_API_KEY"] = ''

## Initialize ChatGPT

In [3]:
chat = ChatOpenAI(temperature=0)

## Initialize attributes

AWA2 attributes from file predicates.txt in https://cvml.ist.ac.at/AwA2/

In [4]:
attributes = [
    "black", "small", "blue", "brown", "gray", "toughskin", "red", "yellow",
    "patches", "spots", "stripes", "orange",
    "furry", "hairless",
    "big",  "white",
    "bulbous", "lean", "flippers", "hands", "hooves", "pads", "paws", "longleg", "longneck", "tail"
    "chewteeth", "meatteeth", "buckteeth", "strainteeth", "horns", "claws", "tusks",
    "smelly", "flys", "hops",
    "swims", "tunnels", "walks",
    "fast", "slow",
    "strong", "weak", "muscle",
    "bipedal", "quadrapedal",
    "active", "inactive", "nocturnal", "hibernate", "agility",
    "fish", "meat", "plankton", "vegetation", "insects",
    "forager", "grazer", "hunter", "scavenger", "skimmer", "stalker",
    "newworld", "oldworld",
    "arctic", "coastal", "desert", "bush", "plains", "forest", "fields", "jungle", "mountains", "ocean", "ground", "water", "tree", "cave",
    "fierce", "timid", "smart",
    "group", "solitary", "nestspot",
    "domestic"
]

## Generate attribute types

### Design examples for few-shot prompt
examples are a list of example dicts. Each example contains example attributes and their corresponding attribute types. These are the examples used fot context learning to let LLM understand the task and the output format.

In [5]:
examples = [
    {
        "example_attrs": "black, spots, small, fast, hibernate, fish, forager",
        "example_types": {
            "black": "color",
            "spots": "pattern",
            "small": "size",
            "fast": "speed",
            "hibernate": "habit",
            "fish": "diet",
            "forager": "feeding behavior"
        }
    }
]

### Design few-shot prompt for generating attribute types
Design a message prompt template containing ***a system message*** (task instruction), a example ***question-answer pair*** (few shot prompt) and the ***final true question***.

In [6]:
system_message_template = system_message_template = """
You are a helpful animal-relevant attribute type annotator that given you some attributes you need to give each of them one attribute type.
The output format is a json string, key is the attribute, value is the attribute type."""

example_human_template = "Attributes: {example_attrs}."
example_ai_template = "Answer: {example_types}."
question_template = "Attributes: {raw_attrs}."

system_message_prompt = SystemMessagePromptTemplate.from_template(system_message_template)
example_human = HumanMessagePromptTemplate.from_template(example_human_template)
example_ai = AIMessagePromptTemplate.from_template(example_ai_template)
human_question = HumanMessagePromptTemplate.from_template(question_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_human, example_ai, human_question]
)


### Turn few-shot prompt into message format
Embed an example into disigned message prompt template forms a message prompt, which is the input of LLM.

In [7]:
example = examples[0]
chat_prompt = chat_prompt.format_messages(
    example_attrs=example['example_attrs'],
    example_types=example['example_types'],
    raw_attrs=", ".join(attributes))
chat_prompt

[SystemMessage(content='\nYou are a helpful animal-relevant attribute type annotator that given you some attributes you need to give each of them one attribute type.\nThe output format is a json string, key is the attribute, value is the attribute type.', additional_kwargs={}),
 HumanMessage(content='Attributes: black, spots, small, fast, hibernate, fish, forager.', additional_kwargs={}, example=False),
 AIMessage(content="Answer: {'black': 'color', 'spots': 'pattern', 'small': 'size', 'fast': 'speed', 'hibernate': 'habit', 'fish': 'diet', 'forager': 'feeding behavior'}.", additional_kwargs={}, example=False),
 HumanMessage(content='Attributes: black, small, blue, brown, gray, toughskin, red, yellow, patches, spots, stripes, orange, furry, hairless, big, white, bulbous, lean, flippers, hands, hooves, pads, paws, longleg, longneck, tailchewteeth, meatteeth, buckteeth, strainteeth, horns, claws, tusks, smelly, flys, hops, swims, tunnels, walks, fast, slow, strong, weak, muscle, bipedal, 

### Generate attribute types with ChatGPT

In [8]:
output = chat(chat_prompt).content
output

'Answer: \n{\n  "black": "color",\n  "small": "size",\n  "blue": "color",\n  "brown": "color",\n  "gray": "color",\n  "toughskin": "physical characteristic",\n  "red": "color",\n  "yellow": "color",\n  "patches": "pattern",\n  "spots": "pattern",\n  "stripes": "pattern",\n  "orange": "color",\n  "furry": "physical characteristic",\n  "hairless": "physical characteristic",\n  "big": "size",\n  "white": "color",\n  "bulbous": "physical characteristic",\n  "lean": "physical characteristic",\n  "flippers": "physical characteristic",\n  "hands": "physical characteristic",\n  "hooves": "physical characteristic",\n  "pads": "physical characteristic",\n  "paws": "physical characteristic",\n  "longleg": "physical characteristic",\n  "longneck": "physical characteristic",\n  "tailchewteeth": "physical characteristic",\n  "meatteeth": "physical characteristic",\n  "buckteeth": "physical characteristic",\n  "strainteeth": "physical characteristic",\n  "horns": "physical characteristic",\n  "claws"

Change the output from a string to the dict format

In [9]:
prefix = "Answer: "
if prefix in output:
    processed_out = output.split(prefix)[1]
    type_dict = json.loads(processed_out)
    pprint(type_dict)
else:
    print("format error")

{'active': 'activity level',
 'agility': 'physical characteristic',
 'arctic': 'habitat',
 'big': 'size',
 'bipedal': 'movement',
 'black': 'color',
 'blue': 'color',
 'brown': 'color',
 'buckteeth': 'physical characteristic',
 'bulbous': 'physical characteristic',
 'bush': 'habitat',
 'cave': 'habitat',
 'claws': 'physical characteristic',
 'coastal': 'habitat',
 'desert': 'habitat',
 'domestic': 'behavior',
 'fast': 'speed',
 'fields': 'habitat',
 'fierce': 'behavior',
 'fish': 'diet',
 'flippers': 'physical characteristic',
 'flys': 'movement',
 'forager': 'feeding behavior',
 'forest': 'habitat',
 'furry': 'physical characteristic',
 'gray': 'color',
 'grazer': 'feeding behavior',
 'ground': 'habitat',
 'group': 'social behavior',
 'hairless': 'physical characteristic',
 'hands': 'physical characteristic',
 'hibernate': 'habit',
 'hooves': 'physical characteristic',
 'hops': 'movement',
 'horns': 'physical characteristic',
 'hunter': 'feeding behavior',
 'inactive': 'activity level

Initialize hierarchy

In [10]:
attribute_types = list(set(type_dict.values()))
hierarchy = {at:[] for at in attribute_types}
hierarchy

{'strength': [],
 'speed': [],
 'activity level': [],
 'activity pattern': [],
 'habitat': [],
 'behavior': [],
 'size': [],
 'pattern': [],
 'diet': [],
 'odor': [],
 'geographical region': [],
 'physical characteristic': [],
 'intelligence': [],
 'feeding behavior': [],
 'color': [],
 'social behavior': [],
 'habit': [],
 'movement': []}

In [11]:
for k, v in type_dict.items():
    hierarchy[v].append(k)
pprint(hierarchy)

{'activity level': ['active', 'inactive'],
 'activity pattern': ['nocturnal'],
 'behavior': ['fierce', 'timid', 'nestspot', 'domestic'],
 'color': ['black',
           'blue',
           'brown',
           'gray',
           'red',
           'yellow',
           'orange',
           'white'],
 'diet': ['fish', 'meat', 'plankton', 'vegetation', 'insects'],
 'feeding behavior': ['forager',
                      'grazer',
                      'hunter',
                      'scavenger',
                      'skimmer',
                      'stalker'],
 'geographical region': ['newworld', 'oldworld'],
 'habit': ['hibernate'],
 'habitat': ['arctic',
             'coastal',
             'desert',
             'bush',
             'plains',
             'forest',
             'fields',
             'jungle',
             'mountains',
             'ocean',
             'ground',
             'water',
             'tree',
             'cave'],
 'intelligence': ['smart'],
 'movement': ['flys

## Building hierarchy for an attribute type

There can be more hierarchy relation in physical characteristic. For example, ***tailchewteeth***, ***meatteeth***, ***buckteeth***, ***strainteeth*** are all teeth types. Therefore, we can continue to optimize the hierarchical annotation.

### Initial annotation working space
This script is executed on colab, if you run in local, please change work space to your own local dir. The work space will store two types of json files. One is the hierarchy of one attribute type generated by LLM, the other is the correction result by human expert.

In [12]:
iter_no = 0
step = 10
work_space = "cache/"
if not os.path.exists(work_space):
    os.mkdir(work_space)
llm_file_name_template = "llm_{attribute_type}_iter_{iter_no}.json"
expert_correct_file_name_template = "exp_{attribute_type}_iter_{iter_no}.json"

In [13]:
attribute_type = 'physical characteristic'
attributes = hierarchy['physical characteristic']
attributes

['toughskin',
 'furry',
 'hairless',
 'bulbous',
 'lean',
 'flippers',
 'hands',
 'hooves',
 'pads',
 'paws',
 'longleg',
 'longneck',
 'tailchewteeth',
 'meatteeth',
 'buckteeth',
 'strainteeth',
 'horns',
 'claws',
 'tusks',
 'muscle',
 'agility']

### Design examples for few-shot prompt
Similar to former few shot example, we design example for adding new attributes into the hierarchy. The attribute hierarchy before adding is store in the field ***example_before_grow***, the attributes waited to be added stored in the field ***example_attrs***, the example attribute hierarchy after updating is stored in ***example_after_grow***

In [14]:
example = {
    "example_before_grow": {
        "teeth": {
            "meatteeth": {}
        },
    },
    "example_attrs": "hairless, toughskin, buckteeth, agility",
    "example_after_grow": {
        "teeth": {
            "meatteeth": {},
            "buckteeth": {}
        },
        "agility": {},
        "skin": {
            "toughskin": {}
        },
        "hair": {
            "hairless": {}
        }
    }
}

"""Remove attributes in above example"""
for attr in example['example_attrs'].split(', '):
    attributes.remove(attr)
attributes

['furry',
 'bulbous',
 'lean',
 'flippers',
 'hands',
 'hooves',
 'pads',
 'paws',
 'longleg',
 'longneck',
 'tailchewteeth',
 'meatteeth',
 'strainteeth',
 'horns',
 'claws',
 'tusks',
 'muscle']

### Design prompt template for building hierarchy of one attribute type
Similar to former message prompt, this message prompt also contains ***a system message*** (task instruction), a example ***question-answer pair*** (few shot example) and the ***final true question***.

In [15]:
system_message_template = """You are a helpful animal-relevant attribute tree annotator that given you an attribute tree you can add new attributes into the tree. Please return answer using json string.
The attribute models the hierarchical relation among attributes."""
example_human_template = "The attribute tree is {example_before_grow}. You should add these attributes into the tree: {example_attrs}."
example_ai_template = "Answer: {example_after_grow}"
question_template = "The attribute tree is {example_after_grow}. You should add these attributes into the tree: {raw_attrs}."

In [16]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message_template)
example_human_prompt = HumanMessagePromptTemplate.from_template(example_human_template)
example_ai_prompt = AIMessagePromptTemplate.from_template(example_ai_template)
human_question_prompt = HumanMessagePromptTemplate.from_template(question_template)

chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_human_prompt, example_ai_prompt, human_question_prompt]
)

### Generate prompt for current iteration (**iteration start point**)
Each iteration embedding a certain number of new attributes and example into the message prompt.

In [21]:
raw_attrs = attributes[iter_no * step: (iter_no + 1) * step]
if (iter_no + 1) * step > len(raw_attrs):
    print("All attributes added finished!")

print(f"\nIteration {iter_no}: \nattributes waited to be added: {raw_attrs}")

chat_prompt = chat_prompt_template.format_messages(
    example_before_grow=json.dumps(example['example_before_grow']),
    example_attrs=json.dumps(example['example_attrs']),
    example_after_grow=json.dumps(example['example_after_grow']),
    raw_attrs=", ".join(raw_attrs))
chat_prompt

All attributes added finished!

Iteration 1: 
attributes waited to be added: ['tailchewteeth', 'meatteeth', 'strainteeth', 'horns', 'claws', 'tusks', 'muscle']


[SystemMessage(content='You are a helpful animal-relevant attribute tree annotator that given you an attribute tree you can add new attributes into the tree. Please return answer using json string.\nThe attribute models the hierarchical relation among attributes.', additional_kwargs={}),
 HumanMessage(content='The attribute tree is {"teeth": {"meatteeth": {}, "buckteeth": {}}, "agility": {}, "skin": {"toughskin": {}}, "hair": {"hairless": {}}}. You should add these attributes into the tree: ["furry", "bulbous", "lean", "flippers", "hands", "hooves", "pads", "paws", "longleg", "longneck"].', additional_kwargs={}, example=False),
 AIMessage(content='Answer: {"teeth": {"meatteeth": {}, "buckteeth": {}}, "agility": {}, "skin": {"toughskin": {}}, "hair": {"hairless": {}, "furry": {}}, "body": {"bulbous": {}, "lean": {}}, "limbs": {"flippers": {}, "hands": {}, "hooves": {}, "feet": {"paws": {"pads": {}}}, "legs": {"longleg": {}}}, "neck": {"longneck": {}}}', additional_kwargs={}, example=Fal

### Update hierarchy

In [22]:
output = chat(chat_prompt).content
output

'Answer: {"teeth": {"meatteeth": {"tailchewteeth": {}, "strainteeth": {}}, "buckteeth": {}}, "agility": {}, "skin": {"toughskin": {}}, "hair": {"hairless": {}, "furry": {}}, "body": {"bulbous": {}, "lean": {}, "muscle": {}}, "limbs": {"flippers": {}, "hands": {"claws": {}}, "hooves": {}, "feet": {"paws": {"pads": {}}}, "legs": {"longleg": {}}}, "neck": {"longneck": {}}, "horns": {}, "tusks": {}}'

### Post-process the answer
Change the output from a string to the dict format and store the generate result into two same json file. One for recording the answer of LLM in this iteration, another for human expert correction.

In [23]:
prefix = "Answer: "

if prefix in output:
    processed_out = output.split(prefix)[1]
    type_hierarchy = json.loads(processed_out)
    pprint(type_hierarchy)
    llm_file_name = llm_file_name_template.format(attribute_type=attribute_type.replace(' ', '_'), iter_no=iter_no)
    expert_correct_file_name = expert_correct_file_name_template.format(attribute_type=attribute_type.replace(' ', '_'), iter_no=iter_no)

    with open(os.path.join(work_space, llm_file_name), 'w') as f:
        json.dump(type_hierarchy, f, indent=4)

    with open(os.path.join(work_space, expert_correct_file_name), 'w') as f:
        json.dump(type_hierarchy, f, indent=4)

else:
    print("format error")

{'agility': {},
 'body': {'bulbous': {}, 'lean': {}, 'muscle': {}},
 'hair': {'furry': {}, 'hairless': {}},
 'horns': {},
 'limbs': {'feet': {'paws': {'pads': {}}},
           'flippers': {},
           'hands': {'claws': {}},
           'hooves': {},
           'legs': {'longleg': {}}},
 'neck': {'longneck': {}},
 'skin': {'toughskin': {}},
 'teeth': {'buckteeth': {},
           'meatteeth': {'strainteeth': {}, 'tailchewteeth': {}}},
 'tusks': {}}


### Human expert correction (***iteration end point***)

If there is some error make by ChatGPT, we will need human expert to correct it, please ***first*** correct the hierarchy store in the `/content/cache/exp_{attribute_type}_iter{iter_no}.json` file, e.g., /content/cache/exp_physical_characteristic_iter_0.json. Before executing following scripts, the expert should correct the LLM annotation first.

Click it, it will open at sider bar and correct the annotation. After correction, the following script updates the correct hierarchy as a new example for next annotation iteration's few shot learning.



In [25]:
"""load the correct hierarchy"""
with open(os.path.join(work_space, expert_correct_file_name), 'r') as f:
    example_after_grow = json.load(f)

example = dict(
    example_before_grow = example["example_after_grow"],
    example_attrs = raw_attrs,
    example_after_grow = example_after_grow
)

iter_no += 1
print(f"Update example for iteration {iter_no}: ")
pprint(example)

Update example for iteration 2: 
{'example_after_grow': {'agility': {},
                        'body': {'bulbous': {}, 'lean': {}, 'muscle': {}},
                        'hair': {'furry': {}, 'hairless': {}},
                        'horns': {},
                        'limbs': {'feet': {'paws': {'claws': {}, 'pads': {}}},
                                  'flippers': {},
                                  'hands': {},
                                  'hooves': {},
                                  'legs': {'longleg': {}}},
                        'neck': {'longneck': {}},
                        'skin': {'toughskin': {}},
                        'teeth': {'buckteeth': {},
                                  'meatteeth': {},
                                  'strainteeth': {},
                                  'tailchewteeth': {},
                                  'tusks': {}}},
 'example_attrs': ['tailchewteeth',
                   'meatteeth',
                   'strainteeth',
       

**You should back to the iteration start point to keep adding new attributes!**

Finish Building Hierarchy

In [26]:
hierarchy[attribute_type] = example_after_grow
hierarchy

{'strength': ['strong', 'weak'],
 'speed': ['fast', 'slow'],
 'activity level': ['active', 'inactive'],
 'activity pattern': ['nocturnal'],
 'habitat': ['arctic',
  'coastal',
  'desert',
  'bush',
  'plains',
  'forest',
  'fields',
  'jungle',
  'mountains',
  'ocean',
  'ground',
  'water',
  'tree',
  'cave'],
 'behavior': ['fierce', 'timid', 'nestspot', 'domestic'],
 'size': ['small', 'big'],
 'pattern': ['patches', 'spots', 'stripes'],
 'diet': ['fish', 'meat', 'plankton', 'vegetation', 'insects'],
 'odor': ['smelly'],
 'geographical region': ['newworld', 'oldworld'],
 'physical characteristic': {'teeth': {'meatteeth': {},
   'buckteeth': {},
   'tailchewteeth': {},
   'strainteeth': {},
   'tusks': {}},
  'agility': {},
  'skin': {'toughskin': {}},
  'hair': {'hairless': {}, 'furry': {}},
  'body': {'bulbous': {}, 'lean': {}, 'muscle': {}},
  'limbs': {'flippers': {},
   'hands': {},
   'hooves': {},
   'feet': {'paws': {'pads': {}, 'claws': {}}},
   'legs': {'longleg': {}}},
  

In [27]:
for k, v in hierarchy.items():
    if isinstance(v, list):
        hierarchy[k] = {attr:{} for attr in v}

pprint(hierarchy)

{'activity level': {'active': {}, 'inactive': {}},
 'activity pattern': {'nocturnal': {}},
 'behavior': {'domestic': {}, 'fierce': {}, 'nestspot': {}, 'timid': {}},
 'color': {'black': {},
           'blue': {},
           'brown': {},
           'gray': {},
           'orange': {},
           'red': {},
           'white': {},
           'yellow': {}},
 'diet': {'fish': {},
          'insects': {},
          'meat': {},
          'plankton': {},
          'vegetation': {}},
 'feeding behavior': {'forager': {},
                      'grazer': {},
                      'hunter': {},
                      'scavenger': {},
                      'skimmer': {},
                      'stalker': {}},
 'geographical region': {'newworld': {}, 'oldworld': {}},
 'habit': {'hibernate': {}},
 'habitat': {'arctic': {},
             'bush': {},
             'cave': {},
             'coastal': {},
             'desert': {},
             'fields': {},
             'forest': {},
             'ground': {

In [28]:
"""Save the final hierarchy: awa2_hierarchy.json"""
with open(os.path.join(work_space, "awa2_hierarchy.json"), 'w') as f:
        json.dump(hierarchy, f, indent=4)
"All Finished!"

'All Finished!'